
#  高性能プログラミングと性能測定(3) --- 練習問題 (マルチコア, GPU)

# 1. 環境設定
* Jupyter上でコンパイラを起動する, およびジョブ投入を簡便にするための設定
* これは各Jupyterノートブックごとに行う
* 同じノートブックでもログアウトしたりカーネルを再スタートしたときなどは失われるのでそのたびに行うこと

## 1-1. コンパイラ
* この演習環境では, 同じコンパイラでCPUもGPUもサポートしているという理由で, NVIDIA HPC SDKを使う
* コマンド名:
  * C: `nvc`
  * C++: `nvc++`
* コンパイルオプション:
  * `-mp=multicore` をつけると CPU用のOpenMPがサポートされる
  * `-mp=gpu` をつけると GPU用のOpenMPがサポートされる (次週)
* 上記のコマンドを実行できるようにするために, 以下を実行する
  * なお以下はコマンドライン端末上では `module load nvidia` とするのが本来のやり方だがJupyter上で`module`コマンドが動かないのでやむなく以下のようにする

In [ ]:
import os
paths = os.environ["PATH"].split(":")
nvc_path="/work/opt/local/x86_64/cores/nvidia/23.3/Linux_x86_64/23.3/compilers/bin"
if nvc_path not in paths:
    os.environ["PATH"] = ":".join([nvc_path] + paths)


## 1-2. ジョブ投入を簡便に行うスクリプト

In [ ]:
import sys
submit_path = "/work/gt47/share/taura/computational-science-code/00submit"
if submit_path not in sys.path:
    sys.path.append(submit_path)
import submit


# 2. 1/8球の体積 を求めるプログラム
* 以下は3次元の1/8球 $B$ ($x^2 + y^2 + z^2 \leq 1$のうち, $x, y, z \geq 0$の部分)の体積を
$$ \int_B dx dy dz $$
によって求めるプログラムである

In [ ]:
%%writefile omp_ball_base.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>
#include <unistd.h>

double volume_of_ball(long n, int nteams, int nthreads) {
  double h = 1.0 / (double)n;
  long s = 0;
  for (long i = 0; i < n; i++) {
    for (long j = 0; j < n; j++) {
      for (long k = 0; k < n; k++) {
        double x = (i + 0.5) * h;
        double y = (j + 0.5) * h;
        double z = (k + 0.5) * h;
        s += (x * x + y * y + z * z < 1.0);
      }
    }
  }
  return s * h * h * h;
}

int main(int argc, char ** argv) {
  long n           = (1 < argc ? atol(argv[1]) : 100);
  char * nteams_   = getenv("OMP_NUM_TEAMS");
  int    nteams    = (nteams_   ? atoi(nteams_) : 1);
  char * nthreads_ = getenv("OMP_NUM_THREADS");
  int    nthreads  = (nthreads_ ? atoi(nthreads_) : 1);

  printf("n             : %ld\n", n);
  printf("nteams        : %d\n", nteams);
  printf("nthreads      : %d\n", nthreads);
  /* 計測開始 */
  double t0 = omp_get_wtime();
  /* 計算本体 */
  double v = volume_of_ball(n, nteams, nthreads);
  /* 計測終了 */
  double t1 = omp_get_wtime();
  double dt = t1 - t0;          /* sec */
  double error = fabs(v - M_PI/6.0);
  if (error > 1.0e-2) {
    fprintf(stderr, "WARNING: error (%f) > 0.01\n", error);
    fprintf(stderr, "check your program\n");
  }
  printf("volume        : %.9f\n", v);
  printf("error         : %e\n", error);
  printf("elapsed       : %7.3f\n", dt);
  printf("n^3 / nsec    : %7.3f\n",
         (double)n * (double)n * (double)n / dt * 1.0e-9);
  return 0;
}

In [ ]:
%%bash
nvc -fast omp_ball_base.c -o omp_ball_base.exe

* 第一引数(以下の512)は積分区間を何等分するかを決める
* 大きくすると精度と計算量が上がる(第一引数$=n$として$n^3$に比例)
* 答えは当然ながら
$$ \frac{1}{8} \times \frac{4}{3}\pi = \frac{\pi}{6} $$
の近似値になる
* 以下の512を変えて精度(error)と実行時間が変わることを確認せよ

In [ ]:
%%bash_submit
./omp_ball_base.exe 512


# <font color="green"> Problem 1 :  マルチコア並列化</font>
* 上記をOpenMPを使ってマルチコアCPU用に並列化せよ


In [ ]:
%%writefile omp_ball_mp.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>
#include <unistd.h>

double volume_of_ball(long n, int nteams, int nthreads) {
  double h = 1.0 / (double)n;
  long s = 0;
  for (long i = 0; i < n; i++) {
    for (long j = 0; j < n; j++) {
      for (long k = 0; k < n; k++) {
        double x = (i + 0.5) * h;
        double y = (j + 0.5) * h;
        double z = (k + 0.5) * h;
        s += (x * x + y * y + z * z < 1.0);
      }
    }
  }
  return s * h * h * h;
}

int main(int argc, char ** argv) {
  long n           = (1 < argc ? atol(argv[1]) : 100);
  char * nteams_   = getenv("OMP_NUM_TEAMS");
  int    nteams    = (nteams_   ? atoi(nteams_) : 1);
  char * nthreads_ = getenv("OMP_NUM_THREADS");
  int    nthreads  = (nthreads_ ? atoi(nthreads_) : 1);

  printf("n             : %ld\n", n);
  printf("nteams        : %d\n", nteams);
  printf("nthreads      : %d\n", nthreads);
  /* 計測開始 */
  double t0 = omp_get_wtime();
  /* 計算本体 */
  double v = volume_of_ball(n, nteams, nthreads);
  /* 計測終了 */
  double t1 = omp_get_wtime();
  double dt = t1 - t0;          /* sec */
  double error = fabs(v - M_PI/6.0);
  if (error > 1.0e-2) {
    fprintf(stderr, "WARNING: error (%f) > 0.01\n", error);
    fprintf(stderr, "check your program\n");
  }
  printf("volume        : %.9f\n", v);
  printf("error         : %e\n", error);
  printf("elapsed       : %7.3f\n", dt);
  printf("n^3 / nsec    : %7.3f\n",
         (double)n * (double)n * (double)n / dt * 1.0e-9);
  return 0;
}

In [ ]:
%%bash
nvc -fast -mp=multicore omp_ball_mp.c -o omp_ball_mp.exe

* 以下の`OMP_NUM_THREADS=1` を色々変えて実行してみよ

In [ ]:
%%bash_submit
OMP_NUM_THREADS=1 ./omp_ball_mp.exe 512

* 以下を適切に修正して様々なスレッド数で実行し, 

In [ ]:
%%bash_submit
#PJM -L gpu=4
#PJM --omp thread=36
#PJM -L elapse=0:01:00
for th in 1 2 適切にスレッド数を並べよ ; do
  echo "====="
  OMP_PROC_BIND=true OMP_NUM_THREADS=${th} ./omp_ball_mp.exe 512
done

* 結果を以下にコピペして性能向上を可視化せよ(性能向上がすぐに頭打ちになるようであれば, $n$の値を調節せよ)

In [ ]:
import re
import matplotlib.pyplot as plt

DATA = r"""
=====
n             : 512
nteams        : 1
nthreads      : 1
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.885
=====
n             : 512
nteams        : 1
nthreads      : 2
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.876
=====
n             : 512
nteams        : 1
nthreads      : 4
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.882
=====
n             : 512
nteams        : 1
nthreads      : 6
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.871
=====
n             : 512
nteams        : 1
nthreads      : 8
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.882
=====
n             : 512
nteams        : 1
nthreads      : 9
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.877
=====
n             : 512
nteams        : 1
nthreads      : 12
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.047
n^3 / nsec    :   2.883
=====
n             : 512
nteams        : 1
nthreads      : 18
volume        : 0.523606822
error         : 8.046296e-06
elapsed       :   0.046
n^3 / nsec    :   2.888
"""

def put_rec(R, D):
    num_teams = int(D["nteams"])
    num_threads = int(D["nthreads"])
    perf = float(D["n^3 / nsec"])
    R.append((num_teams * num_threads, perf))

def speedup(data):
    data = data.strip().split("\n")
    R = []                      # (nteams * nthreads, n^3/nsec)
    D = None
    pat = re.compile(r"(?P<k>[^:]+?) *: +(?P<v>\d+(\.\d+)?)")
    for line in data:
        if line.strip() == "=====":
            if D is not None:
                put_rec(R, D)
            D = {}
        else:
            m = pat.match(line)
            key = m.group("k")
            val = m.group("v")
            D[key] = val
    if D is not None:
        put_rec(R, D)
    plt.ylabel("n^3/nsec")
    plt.xlabel("num_teams * num_threads")
    R.sort()
    X = [x    for x, perf in R]
    Y = [perf for x, perf in R]
    L = [Y[0]/X[0] * x for x in X]
    plt.plot(X, Y)
    plt.plot(X, L)
    plt.show()

speedup(DATA)


# <font color="green"> Problem 2 :  GPU並列化</font>
* 同じプログラムをOpenMPを使ってGPU用に並列化せよ


In [ ]:
%%writefile omp_ball_gpu.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>
#include <unistd.h>

double volume_of_ball(long n, int nteams, int nthreads) {
  double h = 1.0 / (double)n;
  long s = 0;
  for (long i = 0; i < n; i++) {
    for (long j = 0; j < n; j++) {
      for (long k = 0; k < n; k++) {
        double x = (i + 0.5) * h;
        double y = (j + 0.5) * h;
        double z = (k + 0.5) * h;
        s += (x * x + y * y + z * z < 1.0);
      }
    }
  }
  return s * h * h * h;
}

int main(int argc, char ** argv) {
  long n           = (1 < argc ? atol(argv[1]) : 100);
  char * nteams_   = getenv("OMP_NUM_TEAMS");
  int    nteams    = (nteams_   ? atoi(nteams_) : 1);
  char * nthreads_ = getenv("OMP_NUM_THREADS");
  int    nthreads  = (nthreads_ ? atoi(nthreads_) : 1);

  printf("n             : %ld\n", n);
  printf("nteams        : %d\n", nteams);
  printf("nthreads      : %d\n", nthreads);
  /* 計測開始 */
  double t0 = omp_get_wtime();
  /* 計算本体 */
  double v = volume_of_ball(n, nteams, nthreads);
  /* 計測終了 */
  double t1 = omp_get_wtime();
  double dt = t1 - t0;          /* sec */
  double error = fabs(v - M_PI/6.0);
  if (error > 1.0e-2) {
    fprintf(stderr, "WARNING: error (%f) > 0.01\n", error);
    fprintf(stderr, "check your program\n");
  }
  printf("volume        : %.9f\n", v);
  printf("error         : %e\n", error);
  printf("elapsed       : %7.3f\n", dt);
  printf("n^3 / nsec    : %7.3f\n",
         (double)n * (double)n * (double)n / dt * 1.0e-9);
  return 0;
}

In [ ]:
%%bash
nvc -fast -mp=gpu omp_ball_gpu.c -o omp_ball_gpu.exe

* 以下の`OMP_NUM_THREADS=1`, `OMP_NUM_TEMAS=1` を色々変えてやってみよ
* `./omp_ball_gpu.exe` で `OMP_NUM_THREADS=1` としたときと, `./omp_ball_gpu.exe` で`OMP_NUM_TEAMS=1 OMP_NUM_THREADS=1` としたときの性能の違いに注目
* はじめに`OMP_NUM_TEMAS=1` に固定したまま `OMP_NUM_THREADS=1` を増やし, 性能が頭打ちにになる `OMP_NUM_THREADS`の値を求めよ
* 第一引数($n$)の値は適宜変えて, 実行時間があまりにも短くならないようにせよ

In [ ]:
%%bash_submit
OMP_TARGET_OFFLOAD=MANDATORY OMP_NUM_TEAMS=1 OMP_NUM_THREADS=1 ./omp_ball_gpu.exe 512

* 以下を適切に修正して様々なスレッド数で実行し, 

In [ ]:
%%bash_submit
for th in 1 32 64 適切にスレッド数を並べよ ; do
  echo "====="
  OMP_TARGET_OFFLOAD=MANDATORY OMP_NUM_TEAMS=1 OMP_NUM_THREADS=${th} ./omp_ball_gpu.exe 512
done

* 結果を以下にコピペして性能向上を可視化せよ(性能向上がすぐに頭打ちになるようであれば, $n$の値を調節せよ)

In [ ]:
DATA = r'''
ここにデータを貼り付け(この行は消す)
'''

speedup(DATA)

* 次に`OMP_NUM_THREADS` をその値に固定したまま `OMP_NUM_TEMAS` を増やして性能の向上を観察せよ
* 以下の$n$も必要ならば適切な値に修正せよ

In [ ]:
%%bash_submit
OMP_TARGET_OFFLOAD=MANDATORY OMP_NUM_TEAMS=1 OMP_NUM_THREADS=適切なスレッド数 ./omp_ball_gpu.exe 512

* 以下を適切に修正して様々なチーム数で実行し, 

In [ ]:
%%bash_submit
th=適切なスレッド数
for tm in 1 2 3 適切にチーム数を並べよ ; do
  echo "====="
  OMP_TARGET_OFFLOAD=MANDATORY OMP_NUM_TEAMS=${tm} OMP_NUM_THREADS=${th} ./omp_ball_gpu.exe 512
done

* 結果を以下にコピペして性能向上を可視化せよ(性能向上がすぐに頭打ちになるようであれば, $n$の値を調節せよ)

In [ ]:
DATA = r'''
ここにデータを貼り付け(この行は消す)
'''

speedup(DATA)

# 3. 行った計算方法に対するいくつかの注
* 今回, 
$$ \int_B dx dy dz $$
を計算するのに, $B$が3次元であるのに合わせて3重ループを用いているがもちろん, 球の体積を求めることだけが目的ならばその必要はない

* 以下のような2重積分を行えば良く, その方が当然ながら計算量が少ない
$$ \int_{B_{xy}} \sqrt{1 - x^2 - y^2} dx dy dz $$

* 一方ここで行った方法は, $B$が複雑な式($x, y, z$のどれについても容易に解けない式)であっても一般に使えるという利点がある. 具体的には
$$ f(x, y, z) \leq 0 $$
の体積を求めるのに, 上記を満たす$x, y, z$のbounding boxがわかりさえすればよい

* そのような場合に,
  1. bounding box内に, 一様に多数の点を生成
  1. それらの点が, $f(x, y, z) \leq 0$ を満たすか否かを調べ, その割合$r$を求める
  1. $f(x, y, z) \leq 0$ の体積 $\approx$ bounding boxの体積 $\times r$

* なお今回はその「多数の点」として, $[0, 1]^3$の3辺をすべて$n$等分し, できた$n^3$個の小立方体の中心を用いた($((i+0.5)/n, (j+0.5)/n, (k+0.5)/n)$)が, 実はそうする必要はなく, 乱数を用いる方法が一般的(**モンテカルロ法による積分**)
